In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 40,
    filter_low_freq = 2
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9068.71it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3761.09it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 1430374.16it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10291.58it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3536.44it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 924927.51it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9601.91it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f63501a2240>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f62c0736080>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.KNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.KNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

KNRM(
  (embedding): Embedding(16675, 100)
  (kernels): ModuleList(
    (0): GaussianKernel()
    (1): GaussianKernel()
    (2): GaussianKernel()
    (3): GaussianKernel()
    (4): GaussianKernel()
    (5): GaussianKernel()
    (6): GaussianKernel()
    (7): GaussianKernel()
    (8): GaussianKernel()
    (9): GaussianKernel()
    (10): GaussianKernel()
    (11): GaussianKernel()
    (12): GaussianKernel()
    (13): GaussianKernel()
    (14): GaussianKernel()
    (15): GaussianKernel()
    (16): GaussianKernel()
    (17): GaussianKernel()
    (18): GaussianKernel()
    (19): GaussianKernel()
    (20): GaussianKernel()
  )
  (out): Linear(in_features=21, out_features=1, bias=True)
)
Trainable params:  1667522


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-255 Loss-0.789]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4565 - normalized_discounted_cumulative_gain@5(0.0): 0.5244 - mean_average_precision(0.0): 0.4855



[Iter-510 Loss-0.458]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5376 - normalized_discounted_cumulative_gain@5(0.0): 0.5969 - mean_average_precision(0.0): 0.5544



[Iter-765 Loss-0.240]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5692 - normalized_discounted_cumulative_gain@5(0.0): 0.6176 - mean_average_precision(0.0): 0.574



[Iter-1020 Loss-0.129]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5618 - normalized_discounted_cumulative_gain@5(0.0): 0.6208 - mean_average_precision(0.0): 0.5705



[Iter-1275 Loss-0.059]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5673 - normalized_discounted_cumulative_gain@5(0.0): 0.6235 - mean_average_precision(0.0): 0.5814



[Iter-1530 Loss-0.034]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5647 - normalized_discounted_cumulative_gain@5(0.0): 0.6227 - mean_average_precision(0.0): 0.5766



[Iter-1785 Loss-0.024]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5426 - normalized_discounted_cumulative_gain@5(0.0): 0.6164 - mean_average_precision(0.0): 0.5638



[Iter-2040 Loss-0.014]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5519 - normalized_discounted_cumulative_gain@5(0.0): 0.6149 - mean_average_precision(0.0): 0.5719



[Iter-2295 Loss-0.009]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5451 - normalized_discounted_cumulative_gain@5(0.0): 0.6163 - mean_average_precision(0.0): 0.5708



[Iter-2550 Loss-0.004]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.549 - normalized_discounted_cumulative_gain@5(0.0): 0.6131 - mean_average_precision(0.0): 0.5685

Cost time: 3627.693918943405s
